In [2]:
import sys
sys.path[0] = '/tensorflow-2.1.0/python3.6'
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [0]:
import tensorflow as tf
import numpy as np
import math
from tensorflow import keras
from tensorflow.keras import layers, Sequential, losses, optimizers, datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
tf.__version__

'2.1.0'

In [0]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(to_categorical(tf.cast(y, dtype=tf.int32), num_classes=100), dtype=tf.int32)
    return x,y

In [65]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/drive/My Drive/Github/CNN/data/train',
        target_size=(224, 224),
        batch_size=128,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/drive/My Drive/Github/CNN/data/test',
        target_size=(224, 224),
        batch_size=128,
        class_mode='categorical')

def my_generator(generator):
    while True:
        data = next(generator)
        x = data[0]
        y = data[1] , data[1], data[1]
        yield x, y

final_generator = my_generator(train_generator)
val_generator = my_generator(validation_generator)


Found 2500 images belonging to 5 classes.
Found 1500 images belonging to 5 classes.


In [0]:
# %matplotlib inline
# plt.figure(figsize=(20,20))
# for i in range(1, 6):
#     plt.subplot(1, 5, i)
#     im = Image.fromarray(x[i])
#     im = im.resize((224, 224), Image.ANTIALIAS)
#     plt.imshow(im)
#     plt.title(f'This image is {y[i]}')
# plt.show()

In [0]:
 class Inception(layers.Layer):
        def __init__(self, filters):
            super(Inception, self).__init__()
            self.path1 = layers.Conv2D(filters[0][0], kernel_size=(1, 1), strides=1, padding='same',\
                              activation='relu')
            self.path2_1 = layers.Conv2D(filters[1][0], kernel_size=(1, 1), strides=1, padding='same',\
                              activation='relu')
            self.path2_2 = layers.Conv2D(filters[1][1], kernel_size=(3, 3), strides=1, padding='same',\
                              activation='relu')
            self.path3_1 = layers.Conv2D(filters[2][0], kernel_size=(1, 1), strides=1, padding='same',\
                              activation='relu')
            self.path3_2 = layers.Conv2D(filters[2][1], kernel_size=(5, 5), strides=1, padding='same',\
                              activation='relu')
            
            self.path4_1 = layers.MaxPool2D(pool_size=(3, 3), strides=1, padding='same')
            self.path4_2 = layers.Conv2D(filters[3][0], kernel_size=(1, 1), strides=1, padding='same',\
                             activation='relu')
        
        def call(self, inputs, training=None):
            out1 = self.path1(inputs)
            out2 = self.path2_1(inputs)
            out2 = self.path2_2(out2)
            out3 = self.path3_1(inputs)
            out3 = self.path3_2(out3)
            out4 = self.path4_1(inputs)
            out4 = self.path4_2(out4)
            return layers.Concatenate(axis=-1)([out1, out2, out3, out4])

In [0]:
class Auxiliary(layers.Layer):
    def __init__(self, filters=[128, 1024, 5]):
        super(Auxiliary, self).__init__()
        self.avgpool = layers.AvgPool2D(pool_size=(5, 5), strides=3, padding='valid')
        self.conv1 = layers.Conv2D(filters[0], kernel_size=(1, 1), strides=1, padding='same',\
                            activation='relu')
        self.flatten1 = layers.Flatten()
        self.dense1 = layers.Dense(filters[1], activation='relu')
        self.drop = layers.Dropout(0.7)
        self.dense2 = layers.Dense(filters[2], activation='softmax')
        
    def call(self, inputs, training=None):
        out = self.avgpool(inputs)
        out = self.conv1(out)
        out = self.flatten1(out)
        out = self.dense1(out)
        out = self.drop(out)
        out = self.dense2(out)
        return out

In [0]:
class InceptionV1(keras.Model):
    def __init__(self):
        super(InceptionV1, self).__init__()
        self.stem = Sequential([
            layers.Conv2D(64, kernel_size=(7, 7), strides=2, padding='same', activation='relu'),
            layers.MaxPool2D(pool_size=(3, 3), strides=2, padding='same'),
            layers.BatchNormalization(),
            
            layers.Conv2D(64, kernel_size=(1, 1), strides=1, padding='same', activation='relu'),
            layers.Conv2D(192, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=(3, 3), strides=2, padding='same'),
            
            #inception3
            Inception(filters=[[64], [96,128], [16, 32], [32]]),
            Inception(filters=[[128], [128,192], [32, 96], [64]]),
            layers.MaxPool2D(pool_size=(3, 3), strides=2, padding='same')
            ])
            
        #inception4
        self.inception4a = Inception(filters=[[192], [96,208], [16, 48], [64]])
        self.inception4b = Inception(filters=[[160], [112,224], [24, 64], [64]])
        self.auxiliary1 = Auxiliary()
        self.inception4c = Inception(filters=[[128], [128,256], [24, 64], [64]])
        self.inception4d = Inception(filters=[[112], [144,288], [32, 64], [64]])
        self.inception4e = Inception(filters=[[256], [160,320], [32, 128], [128]])
        self.auxiliary2 = Auxiliary()
        self.maxpool1 = layers.MaxPool2D(pool_size=(3, 3), strides=2, padding='same')
        #inception5
        self.inception5a = Inception(filters=[[256], [160,320], [32, 128], [128]])
        self.inception5b = Inception(filters=[[384], [192,384], [48, 128], [128]])
        self.avgpool1 = layers.AvgPool2D(pool_size=(7, 7), strides=1, padding='valid')
        self.flatten1 = layers.Flatten()
        self.drop1 = layers.Dropout(0.4)
        self.dense1 = layers.Dense(256, activation='linear')
        self.dense2 = layers.Dense(5, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.stem(inputs)
        x = self.inception4a(x)
        a1 = self.auxiliary1(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        a2 = self.auxiliary2(x)
        x = self.inception4e(x)
        x = self.maxpool1(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool1(x)
        x = self.flatten1(x)
        x = self.drop1(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return [x, a1, a2]

In [72]:
tf.compat.v1.disable_v2_behavior()
epochs = 1000
model = InceptionV1()
model.build(input_shape=(None, 224, 224, 3))
model.summary()
optimizer = optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)
save_best = keras.callbacks.ModelCheckpoint('/drive/My Drive/Github/CNN/InceptionV1_best_model.h5',\
                                               monitor='val_loss', verbose=1, save_best_only=True,\
                                               mode='min')
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, min_delta=0, \
                                              patience=100, mode='auto')
def scheduler(epoch):
    if epoch < 8:
        return 0.1
    else:
        return 0.1 * math.pow(0.96, epoch // 8)
learning_rate = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
callbacks_list = [save_best, early_stop, learning_rate]
model.compile(optimizer=optimizer,
              loss=[losses.categorical_crossentropy, losses.categorical_crossentropy, losses.categorical_crossentropy],
              loss_weights={'output_1': 1.0, 'output_2': 0.3, 'output_3': 0.3},
             metrics=['accuracy'])
history = model.fit(final_generator, epochs=epochs, validation_data=val_generator, steps_per_epoch=6, validation_steps=2,
                       verbose=1, callbacks=callbacks_list, use_multiprocessing=True)

Model: "inception_v1_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_30 (Sequential)   multiple                  677872    
_________________________________________________________________
inception_272 (Inception)    multiple                  376176    
_________________________________________________________________
inception_273 (Inception)    multiple                  449160    
_________________________________________________________________
auxiliary_60 (Auxiliary)     multiple                  2168965   
_________________________________________________________________
inception_274 (Inception)    multiple                  510104    
_________________________________________________________________
inception_275 (Inception)    multiple                  605376    
_________________________________________________________________
inception_276 (Inception)    multiple              

Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-41:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/tensorflow-2.1.0/python3.6/tensorflow_core/python/keras/utils/data_utils.py", line 924, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "/tensorflow-2.1.0/python3.6/tenso

KeyboardInterrupt: ignored